# Comparação de predição de links com embeddings de nodos baseados em caminhadas aleatórias

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/link-prediction/homogeneous-comparison-link-prediction.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/link-prediction/homogeneous-comparison-link-prediction.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>

Este notebook foi desenvolvido como parte da disciplina de "Análise de Redes Sociais" na faculdade. Ele demonstra a comparação do desempenho de predição de links utilizando embeddings aprendidos por Node2Vec [1], Attri2Vec [2], GraphSAGE [3] e GCN [4] no dataset Cora, sob a mesma configuração de divisão de arestas em treino e teste. Node2Vec e Attri2Vec aprendem ao capturar a similaridade de nós no contexto de caminhadas aleatórias. GraphSAGE e GCN aprendem de forma não supervisionada, representando nós que coocorrem em caminhadas aleatórias curtas de forma próxima no espaço de embedding.

O objetivo é abordar a predição de links como um problema de aprendizado supervisionado em cima das representações/embeddings de nós. Após obter os embeddings, um classificador binário pode ser usado para prever a existência, ou não, de um link entre dois nós do grafo. Diversos hiperparâmetros podem influenciar na obtenção do melhor classificador de links - esta demonstração incorpora a seleção de modelos no pipeline para escolher o melhor operador binário a ser aplicado em um par de embeddings de nós.

O processo está dividido em quatro etapas principais:

1. Obter os embeddings para cada nó
2. Para cada conjunto de hiperparâmetros, treinar um classificador
3. Selecionar o classificador com melhor desempenho
4. Avaliar o classificador selecionado em dados não vistos para validar sua capacidade de generalização

**Referências:**

[1] Node2Vec: Scalable Feature Learning for Networks. A. Grover, J. Leskovec. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2016.

[2] Attributed Network Embedding via Subspace Discovery. D. Zhang, Y. Jie, X. Zhu e C. Zhang. Data Mining and Knowledge Discovery, 2019.

[3] Inductive Representation Learning on Large Graphs. W.L. Hamilton, R. Ying, e J. Leskovec. Neural Information Processing Systems (NIPS), 2017.

[4] Graph Convolutional Networks (GCN): Semi-Supervised Classification with Graph Convolutional Networks. Thomas N. Kipf, Max Welling. International Conference on Learning Representations (ICLR), 2017.


In [1]:
# install StellarGraph if running on Google Colab
#import sys
#if 'google.colab' in sys.modules:
#  %pip install -q stellargraph[demos]==1.3.0b

In [2]:
# verify that we're using the correct version of StellarGraph for this notebook
#import stellargraph as sg

#try:
#    sg.utils.validate_notebook_version("1.3.0b")
#except AttributeError:
#    raise ValueError(
#        f"This notebook requires StellarGraph version 1.3.0b, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
#    ) from None

In [3]:
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
import stellargraph as sg

%matplotlib inline

2024-11-27 19:20:35.630491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 19:20:35.772750: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 19:20:35.775962: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-27 19:20:35.775975: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

## Carregamento do conjunto de dados

O dataset Cora é uma rede homogênea onde todos os nós representam artigos, e as arestas entre os nós são links de citação, por exemplo, o artigo A cita o artigo B.

(See [the "Loading from Pandas" demo](../basics/loading-pandas.ipynb) for details on how data can be loaded.)

In [4]:
#dataset = datasets.Cora()
#display(HTML(dataset.description))
#graph, _ = dataset.load(largest_connected_component_only=True, str_node_ids=True)

In [5]:
#import pickle
#import stellargraph as sg

## Carregue o grafo do arquivo
#with open("meu_grafo.gpickle", "rb") as f:
#    G = pickle.load(f)
    
#graph = sg.StellarGraph.from_networkx(G)
#graph.info()

In [6]:
G = nx.read_graphml('../graphs/coauthorship_graph.xml', node_type=str)
graph = sg.StellarGraph.from_networkx(G)

In [7]:
len(G.nodes)

3307

In [8]:
len(G.edges)

7748

In [9]:
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 3307, Edges: 7748

 Node types:
  default: [3307]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [7748]
        Weights: range=[1, 631], mean=4.51962, std=13.8279
        Features: none


## Construir divisões dos dados de entrada

Precisamos dividir os dados cuidadosamente para evitar vazamento de informações e avaliar os algoritmos corretamente:

* Para calcular os embeddings dos nós, um **Grafo de Treinamento** (`graph_train`)
* Para treinar os classificadores, um **Conjunto de Treinamento do Classificador** (`examples_train`) com arestas positivas e negativas que não foram usadas para calcular os embeddings dos nós
* Para escolher o melhor classificador, um **Conjunto de Teste para Seleção de Modelo** (`examples_model_selection`) com arestas positivas e negativas que não foram usadas para calcular os embeddings dos nós ou para treinar o classificador
* Para a avaliação final, com os embeddings dos nós aprendidos a partir do **Grafo de Treinamento** (`graph_train`), o melhor classificador escolhido é aplicado a um **Conjunto de Teste** (`examples_test`) com arestas positivas e negativas que não foram usadas para calcular os embeddings dos nós, treinar o classificador ou selecionar o modelo

### Grafo de Teste

Começamos com o grafo completo e utilizamos a classe `EdgeSplitter` para produzir:

* Grafo de Teste
* Conjunto de teste com exemplos de links positivos/negativos

O Grafo de Teste é o grafo reduzido que obtemos ao remover o conjunto de teste de links do grafo completo.

In [10]:
# Definir um divisor de arestas no grafo original:
edge_splitter_test = EdgeSplitter(graph)

# Amostra aleatoriamente uma fração p=0.1 de todos os links positivos, e o mesmo número de links negativos, do grafo,
# obtendo o grafo reduzido graph_test com os links amostrados removidos:
graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global"
)

print(graph_test.info())

** Sampled 774 positive and 774 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 3307, Edges: 6974

 Node types:
  default: [3307]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [6974]
        Weights: range=[1, 631], mean=4.58589, std=14.2684
        Features: none


### Grafo de Treinamento

Desta vez, utilizamos a classe `EdgeSplitter` no Grafo de Teste e realizamos uma divisão de treino/teste nos exemplos para produzir:

* Grafo de Treinamento
* Conjunto de treinamento com exemplos de links
* Conjunto de exemplos de links para seleção de modelo

In [11]:
# Realizar o mesmo processo para calcular um subconjunto de treinamento a partir do grafo de teste
edge_splitter_train = EdgeSplitter(graph_test)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.1, method="global"
)
(
    examples_train,
    examples_model_selection,
    labels_train,
    labels_model_selection,
) = train_test_split(examples, labels, train_size=0.75, test_size=0.25)

print(graph_train.info())

** Sampled 697 positive and 697 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 3307, Edges: 6277

 Node types:
  default: [3307]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [6277]
        Weights: range=[1, 631], mean=4.56747, std=14.3531
        Features: none


Below is a summary of the different splits that have been created in this section.

In [12]:
pd.DataFrame(
    [
        (
            "Training Set",
            len(examples_train),
            "Train Graph",
            "Test Graph",
            "Train the Link Classifier",
        ),
        (
            "Model Selection",
            len(examples_model_selection),
            "Train Graph",
            "Test Graph",
            "Select the best Link Classifier model",
        ),
        (
            "Test set",
            len(examples_test),
            "Test Graph",
            "Full Graph",
            "Evaluate the best Link Classifier",
        ),
    ],
    columns=("Split", "Number of Examples", "Hidden from", "Picked from", "Use"),
).set_index("Split")

,Number of Examples,Hidden from,Picked from,Use
Split,,,,
Training Set,1045,Train Graph,Test Graph,Train the Link Classifier
Model Selection,349,Train Graph,Test Graph,Select the best Link Classifier model
Test set,1548,Test Graph,Full Graph,Evaluate the best Link Classifier


## Criar o gerador de caminhadas aleatórias

Definimos uma função auxiliar para gerar caminhadas aleatórias enviesadas a partir do grafo fornecido com os parâmetros fixos de caminhada aleatória:

* `p` - Parâmetro da caminhada aleatória "p" que define a probabilidade, "1/p", de retornar ao nó de origem
* `q` - Parâmetro da caminhada aleatória "q" que define a probabilidade, "1/q", de se mover para um nó afastado do nó de origem

In [13]:
from stellargraph.data import BiasedRandomWalk


def create_biased_random_walker(graph, walk_num, walk_length):
    # Configurações de parâmetros para "p" e "q":
    p = 1.0
    q = 1.0
    return BiasedRandomWalk(graph, n=walk_num, length=walk_length, p=p, q=q)

## Configurações de Parâmetros

Treinamos Node2Vec, Attri2Vec, GraphSAGE e GCN seguindo o mesmo procedimento de aprendizado não supervisionado: primeiro geramos um conjunto de caminhadas aleatórias curtas a partir do grafo fornecido e, em seguida, aprendemos os embeddings dos nós a partir de lotes de pares `target, context` coletados das caminhadas aleatórias. Para aprender os embeddings dos nós, é necessário especificar os seguintes parâmetros:

* `dimension` - Dimensionalidade dos embeddings dos nós
* `walk_number` - Número de caminhadas partindo de cada nó
* `walk_length` - Comprimento de cada caminhada aleatória
* `epochs` - Número de épocas para treinar o modelo de aprendizado de embeddings
* `batch_size` - Tamanho do lote para treinar o modelo de aprendizado de embeddings

Definimos consistentemente a dimensão dos embeddings dos nós como 128 para todos os algoritmos. No entanto, usamos diferentes camadas ocultas para aprender os embeddings dos nós de acordo com os algoritmos, a fim de explorar seus respectivos potenciais. Para os demais parâmetros, configuramos os seguintes valores:

|               | Node2Vec | Attri2Vec | GraphSAGE | GCN |
|---------------|----------|-----------|-----------|-----|
| `walk_number` |    20    |     4     |     1     |  1  |
| `walk_length` |     5    |     5     |     5     |  5  |
| `epochs`      |    6     |     6     |     6     |  6  |
| `batch_size`  |    50    |     50    |    50     |  50 |

Como todos os algoritmos utilizam os mesmos valores para `walk_length`, `batch_size` e `epochs`, definimos esses parâmetros de forma uniforme aqui:

In [14]:
walk_length = 5

In [15]:
epochs = 2 # original = 6

In [16]:
batch_size = 50

Para diferentes algoritmos, os usuários podem encontrar a melhor configuração de parâmetros utilizando o conjunto de arestas de ` Model Selection `.

## Node2Vec

Utilizamos o Node2Vec [1] para calcular os embeddings dos nós. Esses embeddings são aprendidos de forma a garantir que nós que estão próximos no grafo permaneçam próximos no espaço de embedding. Treinamos o Node2Vec com os componentes do Node2Vec fornecidos pelo Stellargraph.

In [17]:
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import Node2VecLinkGenerator, Node2VecNodeGenerator
from stellargraph.layer import Node2Vec, link_classification
from tensorflow import keras


def node2vec_embedding(graph, name):

    # Set the embedding dimension and walk number:
    dimension = 128
    walk_number = 20

    print(f"Training Node2Vec for '{name}':")

    graph_node_list = list(graph.nodes())

    # Create the biased random walker to generate random walks
    walker = create_biased_random_walker(graph, walk_number, walk_length)

    # Create the unsupervised sampler to sample (target, context) pairs from random walks
    unsupervised_samples = UnsupervisedSampler(
        graph, nodes=graph_node_list, walker=walker
    )

    # Define a Node2Vec training generator, which generates batches of training pairs
    generator = Node2VecLinkGenerator(graph, batch_size)

    # Create the Node2Vec model
    node2vec = Node2Vec(dimension, generator=generator)

    # Build the model and expose input and output sockets of Node2Vec, for node pair inputs
    x_inp, x_out = node2vec.in_out_tensors()

    # Use the link_classification function to generate the output of the Node2Vec model
    prediction = link_classification(
        output_dim=1, output_act="sigmoid", edge_embedding_method="dot"
    )(x_out)

    # Stack the Node2Vec encoder and prediction layer into a Keras model, and specify the loss
    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    # Train the model
    model.fit(
        generator.flow(unsupervised_samples),
        epochs=epochs,
        verbose=2,
        use_multiprocessing=False,
        workers=4,
        shuffle=True,
    )

    # Build the model to predict node representations from node ids with the learned Node2Vec model parameters
    x_inp_src = x_inp[0]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

    # Get representations for all nodes in ``graph``
    node_gen = Node2VecNodeGenerator(graph, batch_size).flow(graph_node_list)
    node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=0)

    def get_embedding(u):
        u_index = graph_node_list.index(u)
        return node_embeddings[u_index]

    return get_embedding

## Attri2Vec

Utilizamos o Attri2Vec [2] para calcular os embeddings dos nós. O Attri2Vec aprende as representações dos nós realizando um mapeamento linear/não-linear nas características de conteúdo dos nós e, simultaneamente, garantindo que nós que compartilham contextos similares em caminhadas aleatórias tenham representações similares. Como as características de conteúdo dos nós são utilizadas para aprender os embeddings, esperamos que o Attri2Vec alcance um desempenho superior na predição de links em comparação com o Node2Vec, que preserva apenas a estrutura da rede.

In [18]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec


def attri2vec_embedding(graph, name):

    # Set the embedding dimension and walk number:
    dimension = [128]
    walk_number = 4

    print(f"Training Attri2Vec for '{name}':")

    graph_node_list = list(graph.nodes())

    # Create the biased random walker to generate random walks
    walker = create_biased_random_walker(graph, walk_number, walk_length)

    # Create the unsupervised sampler to sample (target, context) pairs from random walks
    unsupervised_samples = UnsupervisedSampler(
        graph, nodes=graph_node_list, walker=walker
    )

    # Define an Attri2Vec training generator, which generates batches of training pairs
    generator = Attri2VecLinkGenerator(graph, batch_size)

    # Create the Attri2Vec model
    attri2vec = Attri2Vec(
        layer_sizes=dimension, generator=generator, bias=False, normalize=None
    )

    # Build the model and expose input and output sockets of Attri2Vec, for node pair inputs
    x_inp, x_out = attri2vec.in_out_tensors()

    # Use the link_classification function to generate the output of the Attri2Vec model
    prediction = link_classification(
        output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
    )(x_out)

    # Stack the Attri2Vec encoder and prediction layer into a Keras model, and specify the loss
    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    # Train the model
    model.fit(
        generator.flow(unsupervised_samples),
        epochs=epochs,
        verbose=2,
        use_multiprocessing=False,
        workers=1,
        shuffle=True,
    )

    # Build the model to predict node representations from node features with the learned Attri2Vec model parameters
    x_inp_src = x_inp[0]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

    # Get representations for all nodes in ``graph``
    node_gen = Attri2VecNodeGenerator(graph, batch_size).flow(graph_node_list)
    node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=0)

    def get_embedding(u):
        u_index = graph_node_list.index(u)
        return node_embeddings[u_index]

    return get_embedding

## GraphSAGE

O GraphSAGE [3] aprende os embeddings dos nós em grafos atribuídos através da agregação das características dos nós vizinhos. Os parâmetros de agregação são aprendidos incentivando pares de nós que coocorrem em caminhadas aleatórias curtas a terem representações similares. Como as características dos nós também são aproveitadas, espera-se que o GraphSAGE apresente um desempenho superior ao Node2Vec na predição de links.

In [19]:
from stellargraph.mapper import GraphSAGELinkGenerator, GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE


def graphsage_embedding(graph, name):

    # Set the embedding dimensions, the numbers of sampled neighboring nodes and walk number:
    dimensions = [128, 128]
    num_samples = [10, 5]
    walk_number = 1

    print(f"Training GraphSAGE for '{name}':")

    graph_node_list = list(graph.nodes())

    # Create the biased random walker to generate random walks
    walker = create_biased_random_walker(graph, walk_number, walk_length)

    # Create the unsupervised sampler to sample (target, context) pairs from random walks
    unsupervised_samples = UnsupervisedSampler(
        graph, nodes=graph_node_list, walker=walker
    )

    # Define a GraphSAGE training generator, which generates batches of training pairs
    generator = GraphSAGELinkGenerator(graph, batch_size, num_samples)

    # Create the GraphSAGE model
    graphsage = GraphSAGE(
        layer_sizes=dimensions,
        generator=generator,
        bias=True,
        dropout=0.0,
        normalize="l2",
    )

    # Build the model and expose input and output sockets of GraphSAGE, for node pair inputs
    x_inp, x_out = graphsage.in_out_tensors()

    # Use the link_classification function to generate the output of the GraphSAGE model
    prediction = link_classification(
        output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
    )(x_out)

    # Stack the GraphSAGE encoder and prediction layer into a Keras model, and specify the loss
    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    # Train the model
    model.fit(
        generator.flow(unsupervised_samples),
        epochs=epochs,
        verbose=2,
        use_multiprocessing=False,
        workers=4,
        shuffle=True,
    )

    # Build the model to predict node representations from node features with the learned GraphSAGE model parameters
    x_inp_src = x_inp[0::2]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

    # Get representations for all nodes in ``graph``
    node_gen = GraphSAGENodeGenerator(graph, batch_size, num_samples).flow(
        graph_node_list
    )
    node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=0)

    def get_embedding(u):
        u_index = graph_node_list.index(u)
        return node_embeddings[u_index]

    return get_embedding

## GCN

O GCN [4] aprende os embeddings dos nós por meio de convolução em grafos. Tradicionalmente, o GCN depende de rótulos dos nós como supervisão para realizar o treinamento. Aqui, consideramos o cenário de predição de links não supervisionada e tentamos aprender embeddings de nós informativos no GCN, garantindo que nós que coocorrem em caminhadas aleatórias curtas sejam representados de forma próxima, como é feito no treinamento do GraphSAGE.

In [20]:
from stellargraph.mapper import FullBatchLinkGenerator, FullBatchNodeGenerator
from stellargraph.layer import GCN, LinkEmbedding


def gcn_embedding(graph, name):

    # Set the embedding dimensions and walk number:
    dimensions = [128, 128]
    walk_number = 1

    print(f"Training GCN for '{name}':")

    graph_node_list = list(graph.nodes())

    # Create the biased random walker to generate random walks
    walker = create_biased_random_walker(graph, walk_number, walk_length)

    # Create the unsupervised sampler to sample (target, context) pairs from random walks
    unsupervised_samples = UnsupervisedSampler(
        graph, nodes=graph_node_list, walker=walker
    )

    # Define a GCN training generator, which generates the full batch of training pairs
    generator = FullBatchLinkGenerator(graph, method="gcn")

    # Create the GCN model
    gcn = GCN(
        layer_sizes=dimensions,
        activations=["relu", "relu"],
        generator=generator,
        dropout=0.3,
    )

    # Build the model and expose input and output sockets of GCN, for node pair inputs
    x_inp, x_out = gcn.in_out_tensors()

    # Use the dot product of node embeddings to make node pairs co-occurring in short random walks represented closely
    prediction = LinkEmbedding(activation="sigmoid", method="ip")(x_out)
    prediction = keras.layers.Reshape((-1,))(prediction)

    # Stack the GCN encoder and prediction layer into a Keras model, and specify the loss
    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    # Train the model
    batches = unsupervised_samples.run(batch_size)
    for epoch in range(epochs):
        print(f"Epoch: {epoch+1}/{epochs}")
        batch_iter = 1
        for batch in batches:
            samples = generator.flow(batch[0], targets=batch[1], use_ilocs=True)[0]
            [loss, accuracy] = model.train_on_batch(x=samples[0], y=samples[1])
            output = (
                f"{batch_iter}/{len(batches)} - loss:"
                + " {:6.4f}".format(loss)
                + " - binary_accuracy:"
                + " {:6.4f}".format(accuracy)
            )
            if batch_iter == len(batches):
                print(output)
            else:
                print(output, end="\r")
            batch_iter = batch_iter + 1

    # Get representations for all nodes in ``graph``
    embedding_model = keras.Model(inputs=x_inp, outputs=x_out)
    node_embeddings = embedding_model.predict(
        generator.flow(list(zip(graph_node_list, graph_node_list)))
    )
    node_embeddings = node_embeddings[0][:, 0, :]

    def get_embedding(u):
        u_index = graph_node_list.index(u)
        return node_embeddings[u_index]

    return get_embedding

## Treinar e avaliar o modelo de predição de links

Existem algumas etapas envolvidas no uso dos embeddings aprendidos para realizar a predição de links:
1. Calculamos os embeddings de links/arestas para as amostras de arestas positivas e negativas aplicando um operador binário nos embeddings dos nós de origem e destino de cada aresta amostrada.
2. Dado os embeddings dos exemplos positivos e negativos, treinamos um classificador de regressão logística para prever um valor binário indicando se uma aresta entre dois nós deve ou não existir.
3. Avaliamos o desempenho do classificador de links para cada um dos 4 operadores nos dados de treinamento com os embeddings dos nós calculados no **Grafo de Treinamento** (`graph_train`) e selecionamos o melhor classificador.
4. O melhor classificador é então usado para calcular os escores nos dados de teste com os embeddings dos nós treinados no **Grafo de Treinamento** (`graph_train`).

Abaixo estão um conjunto de funções auxiliares que nos permitem repetir essas etapas para cada um dos operadores binários.

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler


# 1. link embeddings
def link_examples_to_features(link_examples, transform_node, binary_operator):
    return [
        binary_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]


# 2. training classifier
def train_link_prediction_model(
    link_examples, link_labels, get_embedding, binary_operator
):
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    clf.fit(link_features, link_labels)
    return clf


def link_prediction_classifier(max_iter=5000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])


# 3. and 4. evaluate classifier
def evaluate_link_prediction_model(
    clf, link_examples_test, link_labels_test, get_embedding, binary_operator
):
    link_features_test = link_examples_to_features(
        link_examples_test, get_embedding, binary_operator
    )
    score = evaluate_roc_auc(clf, link_features_test, link_labels_test)
    return score


def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)

    # check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

Consideramos 4 operadores diferentes:

* *Hadamard*
* $L_1$
* $L_2$
* *average*

O artigo [1] fornece uma descrição detalhada desses operadores. Todos os operadores produzem embeddings de links com a mesma dimensionalidade dos embeddings de entrada dos nós (128 dimensões no nosso exemplo).


In [22]:
def operator_hadamard(u, v):
    return u * v


def operator_l1(u, v):
    return np.abs(u - v)


def operator_l2(u, v):
    return (u - v) ** 2


def operator_avg(u, v):
    return (u + v) / 2.0


def run_link_prediction(binary_operator, embedding_train):
    clf = train_link_prediction_model(
        examples_train, labels_train, embedding_train, binary_operator
    )
    score = evaluate_link_prediction_model(
        clf,
        examples_model_selection,
        labels_model_selection,
        embedding_train,
        binary_operator,
    )

    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }


binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]

### Treinar e avaliar o modelo de links com o embedding especificado

In [23]:
def train_and_evaluate(embedding, name):

    embedding_train = embedding(graph_train, "Train Graph")

    # Train the link classification model with the learned embedding
    results = [run_link_prediction(op, embedding_train) for op in binary_operators]
    best_result = max(results, key=lambda result: result["score"])
    print(
        f"\nBest result with '{name}' embeddings from '{best_result['binary_operator'].__name__}'"
    )
    display(
        pd.DataFrame(
            [(result["binary_operator"].__name__, result["score"]) for result in results],
            columns=("name", "ROC AUC"),
        ).set_index("name")
    )

    # Evaluate the best model using the test set
    test_score = evaluate_link_prediction_model(
        best_result["classifier"],
        examples_test,
        labels_test,
        embedding_train,
        best_result["binary_operator"],
    )

    return test_score

### Coletar os resultados da predição de links para Node2Vec, Attri2Vec, GraphSAGE e GCN

In [24]:
type(graph.nodes()[0])

str

#### Obter o resultado da predição de links com Node2Vec

In [ ]:
node2vec_result = train_and_evaluate(node2vec_embedding, "Node2Vec")

Training Node2Vec for 'Train Graph':
link_classification: using 'dot' method to combine node embeddings into edge embeddings


Epoch 1/2
10039/10039 - 58s - loss: 0.5486 - binary_accuracy: 0.6733 - 58s/epoch - 6ms/step
Epoch 2/2


#### Obter o resultado da predição de links com Attri2Vec

In [ ]:
print(graph_train.node_features())

In [ ]:
attri2vec_result = train_and_evaluate(attri2vec_embedding, "Attri2Vec")

#### Obter o resultado da predição de links com GraphSAGE

In [ ]:
graphsage_result = train_and_evaluate(graphsage_embedding, "GraphSAGE")

#### Obter o resultado da predição de links com GCN

In [ ]:
gcn_result = train_and_evaluate(gcn_embedding, "GCN")

#### Comparação entre Node2Vec, Attri2Vec, GraphSAGE e GCN no conjunto de teste

As pontuações de ROC AUC no conjunto de teste de links para diferentes embeddings com seus respectivos melhores operadores:

In [ ]:
pd.DataFrame(
    [
        ("Node2Vec", node2vec_result),
        ("Attri2Vec", attri2vec_result),
        ("GraphSAGE", graphsage_result),
        ("GCN", gcn_result),
    ],
    columns=("name", "ROC AUC"),
).set_index("name")

## Conclusão

Este exemplo demonstrou como usar a biblioteca `stellargraph` para construir um algoritmo de predição de links para grafos homogêneos utilizando embeddings não supervisionados aprendidos por Node2Vec [1], Attri2Vec [2], GraphSAGE [3] e GCN [4].

Para mais informações sobre o processo de predição de links, todos esses algoritmos possuem demonstrações específicas com mais detalhes:

- [Node2Vec](node2vec-link-prediction.ipynb)
- [Attri2Vec](attri2vec-link-prediction.ipynb)
- [GraphSAGE](graphsage-link-prediction.ipynb)
- [GCN](gcn-link-prediction.ipynb)

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/link-prediction/homogeneous-comparison-link-prediction.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/link-prediction/homogeneous-comparison-link-prediction.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>